In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input, decode_predictions
import nibabel as nib
import torch
from skimage.transform import resize
from skimage.color import gray2rgb
import os

2023-04-16 19:12:33.916212: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-16 19:12:34.932674: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
train_on_gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# Load the pre-trained VGG19 model
base_model = VGG19(include_top=False, weights='imagenet',input_shape=(224, 224, 3))

x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output1 = layers.Dense(1, activation='sigmoid', name='output1')(x)
output2 = layers.Dense(1, activation='sigmoid', name='output2')(x)
model = models.Model(inputs=base_model.input, outputs=[output1, output2])


2023-04-16 19:12:37.502684: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
from tensorflow.keras.optimizers import Adam
if train_on_gpu:
    with tf.device('/GPU:0'):  # specify GPU to use
        model = model
else:
    model = model
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
labels_df =pd.read_csv('/home/sandyap/DM project/cleaned_labels.csv')
with tf.device('/GPU:0'):  # specify GPU to use
        labels_df=labels_df

In [6]:
labels_df = labels_df.rename(columns={'Recognizable-Facial-Feature': 'output1', 'Brain-Feature-Loss': 'output2'})

In [7]:
labels_df.replace({'Yes': 1, 'No': 0}, inplace=True)

In [8]:
train_labels_df, test_labels_df = train_test_split(labels_df, test_size=0.4, random_state=42)
with tf.device('/GPU:0'):  # specify GPU to use
        train_labels_df=train_labels_df
with tf.device('/GPU:0'):  # specify GPU to use
        test_labels_df=test_labels_df

In [9]:
# Loop through each image file in the folder
train_images = []
train_labels = []
k=0
for i in range(len(train_labels_df)):
    filename = train_labels_df.iloc[i]['Filename']
    k=k+1
    print(k)
    print(filename)
    facial_label = train_labels_df.iloc[i]['output1']
    #print('----------------------actual------------------------')
    #print(facial_label)
    brain_label = train_labels_df.iloc[i]['output2']
    #print(brain_label)
    
    # Load the image data
    img = nib.load('/home/sandyap/DM project/files/'+filename+'.gz')
    img_data = img.get_fdata()
    with tf.device('/GPU:0'):  # specify GPU to use
        img_data=img_data
    print('image shape')
    print(img_data.shape)
    img_data = gray2rgb(img_data)
    print(img_data.shape)
    
    # Preprocess the image data
    #img_data = np.expand_dims(img_data, axis=0)
    img_data = resize(img_data[0], (224, 224,3), anti_aliasing=True, anti_aliasing_sigma=0.2)
    print('after resizing')
    print(img_data.shape)
    img_data = preprocess_input(img_data)
    print('after using preprocess_input')
    print(img_data.shape)
    train_images.append(img_data)
    train_labels.append([facial_label, brain_label])
train_images = np.array(train_images)
train_labels = np.array(train_labels)

1
IXI296-HH-1970-T1_bet_7.nii
image shape
(256, 256, 150)
(256, 256, 150, 3)
after resizing
(224, 224, 3)
after using preprocess_input
(224, 224, 3)
2
IXI178-Guys-0778-T1_bse_less_s58_r1.nii
image shape
(256, 256, 150)


/local_scratch/pbs.495768.pbs02/ipykernel_2071948/154135214.py:28: UserWarning: Anti-aliasing standard deviation greater than zero but not down-sampling along all axes
  img_data = resize(img_data[0], (224, 224,3), anti_aliasing=True, anti_aliasing_sigma=0.2)


(256, 256, 150, 3)
after resizing
(224, 224, 3)
after using preprocess_input
(224, 224, 3)
3
IXI461-Guys-0998-T1_bet_79.nii
image shape
(256, 256, 150)
(256, 256, 150, 3)
after resizing
(224, 224, 3)
after using preprocess_input
(224, 224, 3)
4
IXI629-Guys-1095-T1_bet_71.nii
image shape
(256, 256, 150)
(256, 256, 150, 3)
after resizing
(224, 224, 3)
after using preprocess_input
(224, 224, 3)
5
IXI276-HH-1840-T1_bse_default.nii
image shape
(256, 256, 150)
(256, 256, 150, 3)
after resizing
(224, 224, 3)
after using preprocess_input
(224, 224, 3)
6
IXI594-Guys-1089-T1_bse_less_s52_r1.nii
image shape
(256, 256, 150)
(256, 256, 150, 3)
after resizing
(224, 224, 3)
after using preprocess_input
(224, 224, 3)
7
IXI142-Guys-0786-T1_bet_07.nii
image shape
(256, 256, 150)
(256, 256, 150, 3)
after resizing
(224, 224, 3)
after using preprocess_input
(224, 224, 3)
8
IXI360-Guys-0912-T1_bse_less_s47_r1.nii
image shape
(256, 256, 150)
(256, 256, 150, 3)
after resizing
(224, 224, 3)
after using preproc

In [12]:
print(train_images.shape)

(1236, 180, 180, 3)


In [11]:
history=model.fit(train_images, [train_labels[:,0], train_labels[:,1]], epochs=20, batch_size=32)

Epoch 1/20
39/39 [==============================] - 225s 6s/step - loss: 1.3298 - output1_loss: 0.6701 - output2_loss: 0.6597 - output1_accuracy: 0.6456 - output2_accuracy: 0.6351
Epoch 2/20
39/39 [==============================] - 231s 6s/step - loss: 1.3150 - output1_loss: 0.6535 - output2_loss: 0.6616 - output1_accuracy: 0.6481 - output2_accuracy: 0.6367
Epoch 3/20
39/39 [==============================] - 230s 6s/step - loss: 1.3245 - output1_loss: 0.6607 - output2_loss: 0.6638 - output1_accuracy: 0.6456 - output2_accuracy: 0.6327
Epoch 4/20
39/39 [==============================] - 230s 6s/step - loss: 1.3259 - output1_loss: 0.6583 - output2_loss: 0.6676 - output1_accuracy: 0.6456 - output2_accuracy: 0.6367
Epoch 5/20
39/39 [==============================] - 232s 6s/step - loss: 1.3130 - output1_loss: 0.6565 - output2_loss: 0.6565 - output1_accuracy: 0.6472 - output2_accuracy: 0.6400
Epoch 6/20
39/39 [==============================] - 232s 6s/step - loss: 1.3172 - output1_loss: 0.65

In [ ]:
import matplotlib.pyplot as plt

# Get the training and validation loss values from the history object
op1_loss = history.history['output1_loss']
op2_loss = history.history['output2_loss']
op1_acc=history.history['output1_accuracy']
op2_acc=history.history['output2_accuracy']

# Create a plot of the training and validation loss over each epoch
#plt.plot(train_loss, label='Training Loss')
plt.plot(op1_acc, label='output1 accuracy')
plt.plot(op2_acc, label='output2 accuracy')
plt.xlabel('Epoch')
plt.ylabel('accuracy')
plt.title('Epoch vs accuracy')
plt.legend()
plt.show()
plt.plot(op1_loss, label='output1 loss')
plt.plot(op2_loss, label='output2 loss')
plt.title('Epoch vs loss')
plt.legend()
plt.show()


In [ ]:
# Loop through each image file in the folder
test_images = []
test_labels = []
k=1
for i in range(len(test_labels_df)):
    filename = test_labels_df.iloc[i]['Filename']
    k=k+1
    print(i)
    print(filename)
    facial_label = test_labels_df.iloc[i]['output1']
    print('----------------------actual------------------------')
    print(facial_label)
    brain_label = test_labels_df.iloc[i]['output2']
    print(brain_label)
    
    # Load the image data
    img = nib.load('/home/sandyap/DM project/files/'+filename+'.gz')
    img_data = img.get_fdata()
    with tf.device('/GPU:0'):  # specify GPU to use
        img_data=img_data
    print(img_data.shape)
    img_data = gray2rgb(img_data)
    print(img_data.shape)
    
    # Preprocess the image data
    #img_data = np.expand_dims(img_data, axis=0)
    img_data = resize(img_data[0], (224, 224,3), anti_aliasing=True, anti_aliasing_sigma=0.2)
    print('debug1')
    print(img_data.shape)
    img_data = preprocess_input(img_data)
    print('debug2')
    print(img_data.shape)
    
    '''preds = model.predict(img_data)
    print('debug3')
    pred_facial_prob = preds[0][0]
    pred_brain_prob = preds[0][1]
    print('-------------------predicted-----------------')
    print(pred_facial_prob)
    print(pred_brain_prob)
    # Check if the predicted label matches the actual label
    pred_facial_label = 0 if pred_facial_prob >= 0.5 else 1
    pred_brain_label = 0 if pred_brain_prob >= 0.5 else 1
    
    # Check if the predicted label matches the actual label
    if pred_facial_label == facial_label: #and pred_brain_label == brain_label:
        print('Facial features are properly removed in', filename)
    else:
        print('Facial features are not properly removed in', filename)
    if  pred_brain_label == brain_label:
        print('brain voxels are properly removed in', filename)
    else:
        print('brain voxels are not properly removed in', filename)'''
    test_images.append(img_data)
    test_labels.append([facial_label, brain_label])
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
loss, test_loss1,test_loss2, test_acc1, test_acc2 = model.evaluate(test_images, [test_labels[:,0], test_labels[:,1]])
print('Test Accuracy 1:', test_acc1)
print('Test Accuracy 2:', test_acc2)


In [ ]:
model.predict(test_images)

In [19]:
# Loop through each image file in the folder
test_images = []
test_labels = []
k=0
for i in range(len(test_labels_df)):
    filename = test_labels_df.iloc[i]['Filename']
    k=k+1
    print(i)
    print(filename)
    facial_label = test_labels_df.iloc[i]['Recognizable-Facial-Feature']
    print('----------------------actual------------------------')
    print(facial_label)
    brain_label = test_labels_df.iloc[i]['Brain-Feature-Loss']
    print(brain_label)
    
    # Load the image data
    img = nib.load('/home/sandyap/DM project/files/'+filename+'.gz')
    img_data = img.get_fdata()
    with tf.device('/GPU:0'):  # specify GPU to use
        img_data=img_data
    print(img_data.shape)
    img_data = gray2rgb(img_data)
    print(img_data.shape)
    
    # Preprocess the image data
    img_data = np.expand_dims(img_data, axis=0)
    img_data = resize(img_data[0], (224, 224,224), anti_aliasing=True, anti_aliasing_sigma=0.2)
    print('debug1')
    print(img_data.shape)
    img_data = preprocess_input(img_data)
    print('debug2')
    print(img_data.shape)
    
    preds = model.predict(img_data)
    print('debug3')
    pred_facial_prob = preds[0][0]
    pred_brain_prob = preds[0][1]
    print('-------------------predicted-----------------')
    print(pred_facial_prob)
    print(pred_brain_prob)
    # Check if the predicted label matches the actual label
    pred_facial_label = 0 if pred_facial_prob >= 0.5 else 1
    pred_brain_label = 0 if pred_brain_prob >= 0.5 else 1
    
    # Check if the predicted label matches the actual label
    if pred_facial_label == facial_label: #and pred_brain_label == brain_label:
        print('Facial features are properly removed in', filename)
    else:
        print('Facial features are not properly removed in', filename)
    if  pred_brain_label == brain_label:
        print('brain voxels are properly removed in', filename)
    else:
        print('brain voxels are not properly removed in', filename)
    #test_images.append(img_data)
    #test_labels.append([facial_label, brain_label])
#test_images = np.array(test_images)
#test_labels = np.array(test_labels)

0
IXI294-IOP-0868-T1_bse_default.nii
----------------------actual------------------------
1
0
(256, 256, 146)
(256, 256, 146, 3)


/local_scratch/pbs.488240.pbs02/ipykernel_3593610/1247766604.py:27: UserWarning: Anti-aliasing standard deviation greater than zero but not down-sampling along all axes
  img_data = resize(img_data[0], (224, 224,224), anti_aliasing=True, anti_aliasing_sigma=0.2)


debug1
(224, 224, 224, 3)
debug2
(224, 224, 224, 3)
7/7 [==============================] - 1s 204ms/step
debug3
-------------------predicted-----------------
[0.43863118]
[0.43863118]
Facial features are properly removed in IXI294-IOP-0868-T1_bse_default.nii
brain voxels are not properly removed in IXI294-IOP-0868-T1_bse_default.nii
1
IXI045-Guys-0713-T1_bet_72.nii
----------------------actual------------------------
0
1
(256, 256, 150)
(256, 256, 150, 3)
debug1
(224, 224, 224, 3)
debug2
(224, 224, 224, 3)
7/7 [==============================] - 1s 203ms/step
debug3
-------------------predicted-----------------
[0.43863118]
[0.43863118]
Facial features are not properly removed in IXI045-Guys-0713-T1_bet_72.nii
brain voxels are properly removed in IXI045-Guys-0713-T1_bet_72.nii
2
IXI450-Guys-1093-T1_bse_default.nii
----------------------actual------------------------
0
1
(256, 256, 150)
(256, 256, 150, 3)
debug1
(224, 224, 224, 3)
debug2
(224, 224, 224, 3)
7/7 [==========================

KeyboardInterrupt: 

In [ ]:
# Loop through each image file in the folder
train_images = []
train_labels = []
k=0
for i in range(len(train_labels_df)):
    filename = train_labels_df.iloc[i]['Filename']
    k=k+1
    print(i)
    print(filename)
    facial_label = train_labels_df.iloc[i]['Recognizable-Facial-Feature']
    print('----------------------actual------------------------')
    print(facial_label)
    brain_label = train_labels_df.iloc[i]['Brain-Feature-Loss']
    print(brain_label)
    
    # Load the image data
    img = nib.load('/home/sandyap/DM project/files/'+filename+'.gz')
    img_data = img.get_fdata()
    with tf.device('/GPU:0'):  # specify GPU to use
        img_data=img_data
    img_data = gray2rgb(img_data)
    img_data = resize(img_data[0], (224, 224,3), anti_aliasing=True, anti_aliasing_sigma=0.2)
    img_data = preprocess_input(img_data)
    
    
    train_images.append(img_data)
    train_labels.append([facial_label, brain_label])
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [ ]:
for i in range(len(test_labels_df)):   
    # Use ResNet50 to predict the image label
    filename = test_labels_df.iloc[i]['Filename']
    k=k+1
    print(k)
    print(filename)
    facial_label = test_labels_df.iloc[i]['Recognizable-Facial-Feature']
    print('----------------------actual------------------------')
    print(facial_label)
    brain_label = test_labels_df.iloc[i]['Brain-Feature-Loss']
    print(brain_label)
    
    preds = model.predict(test_images)
    #print('debug3')
    pred_facial_prob = preds[0][0]
    print('----------------------predicted------------------------')
    print(pred_facial_prob)
    pred_brain_prob = preds[0][1]
    print(pred_brain_prob)
    
    # Check if the predicted label matches the actual label
    pred_facial_label = 1 if pred_facial_prob >= 0.5 else 0
    pred_brain_label = 1 if pred_brain_prob >= 0.5 else 0
    
    print('---------------------predicted--------------------')
    print(pred_facial_label)
    print(pred_brain_label)
    
    # Check if the predicted label matches the actual label
    '''if pred_facial_label == facial_label: #and pred_brain_label == brain_label:
        print('Facial features are properly removed in', filename)
    else:
        print('Facial features are not properly removed in', filename)
    if  pred_brain_label == brain_label:
        print('brain voxels are properly removed in', filename)
    else:
        print('brain voxels are not properly removed in', filename)'''

In [ ]:
# Loop through each image file in the folder
for i in range(len(test_labels_df)):
    filename = test_labels_df.iloc[i]['Filename']
    facial_label = test_labels_df.iloc[i]['Recognizable-Facial-Feature']
    brain_label = test_labels_df.iloc[i]['Brain-Feature-Loss']
    print(filename)
    
    # Load the image data
    img = nib.load('/home/sandyap/DM project/files/'+filename+'.gz')
    img_data = img.get_fdata()
    print(img_data.shape)
    img_data = gray2rgb(img_data)
    print(img_data.shape)
    
    # Preprocess the image data
    img_data = np.expand_dims(img_data, axis=0)
    img_data = resize(img_data[0], (224, 224,224), anti_aliasing=True, anti_aliasing_sigma=0.2)
    print('debug1')
    print(img_data.shape)
    img_data = preprocess_input(img_data)
    print('debug2')
    print(img_data.shape)
    
    # Use ResNet50 to predict the image label
    preds = model.predict(img_data)
    print('debug3')
    pred_facial_prob = preds[0][0]
    pred_brain_prob = preds[0][1]
    
    # Check if the predicted label matches the actual label
    pred_facial_label = 'yes' if pred_facial_prob >= 0.5 else 'no'
    pred_brain_label = 'yes' if pred_brain_prob >= 0.5 else 'no'
    
    # Check if the predicted label matches the actual label
    if pred_facial_label == facial_label: #and pred_brain_label == brain_label:
        print('Facial features are properly removed in', filename)
    else:
        print('Facial features are not properly removed in', filename)
    if  pred_brain_label == brain_label:
        print('brain voxels are properly removed in', filename)
    else:
        print('brain voxels are not properly removed in', filename)